# Formula 1 World Championship (1950 - 2024) Analysis 

<img src="f1_ferrari3.png" style="width:60%; border-radius:20px;" />

## Environment Setup

#### Import of the packages

In [1]:
%pip install --upgrade google-cloud
%pip install --upgrade google-cloud-monitoring
%pip install --upgrade google-cloud-storage
%pip install --upgrade google-cloud-bigquery
%pip install --upgrade google-cloud-bigquery-storage
%pip install --upgrade pyspark
import pyspark
import findspark
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from google.cloud import bigquery
from pyspark.sql import SparkSession


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


#### Displaying the path of Google Cloud credentials file for authentication

In [46]:
!echo %GOOGLE_APPLICATION_CREDENTIALS%

C:\Users\Alessandro\Downloads\ccbd-exam-2025-risino-1e4ecafc3ece.json


#### Setting Google Cloud Credentials and Initializing BigQuery Client

In [45]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\Alessandro\Downloads\ccbd-exam-2025-risino-1e4ecafc3ece.json"

!echo %GOOGLE_APPLICATION_CREDENTIALS%
client = bigquery.Client()
print('done')

C:\Users\Alessandro\Downloads\ccbd-exam-2025-risino-1e4ecafc3ece.json
done


## Dataset Exploration and Selection

#### Selection of 5 Datasets from GCP

In [4]:
circuits = "SELECT * FROM `formula1.circuits` LIMIT 5"
client.query(circuits).to_dataframe()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,25,galvez,Autódromo Juan y Oscar Gálvez,Buenos Aires,Argentina,-34.6943,-58.4593,8,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Os...
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.9680,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,29,adelaide,Adelaide Street Circuit,Adelaide,Australia,-34.9272,138.6170,58,http://en.wikipedia.org/wiki/Adelaide_Street_C...
3,57,zeltweg,Zeltweg,Styria,Austria,47.2039,14.7478,676,http://en.wikipedia.org/wiki/Zeltweg_Airfield
4,70,red_bull_ring,Red Bull Ring,Spielberg,Austria,47.2197,14.7647,678,http://en.wikipedia.org/wiki/Red_Bull_Ring


In [5]:
constructors = "SELECT * FROM `formula1.constructors` LIMIT 5"
client.query(constructors).to_dataframe()

,constructorId,constructorRef,name,nationality,url
0,67,penske,Penske,American,http://en.wikipedia.org/wiki/Penske_Racing
1,73,parnelli,Parnelli,American,http://en.wikipedia.org/wiki/Parnelli
2,88,eagle,Eagle,American,http://en.wikipedia.org/wiki/Anglo_American_Ra...
3,106,scarab,Scarab,American,http://en.wikipedia.org/wiki/Scarab_(constructor)
4,107,watson,Watson,American,http://en.wikipedia.org/wiki/A.J._Watson


In [6]:
drivers = "SELECT * FROM `formula1.drivers` LIMIT 5"
client.query(drivers).to_dataframe()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,31,montoya,\N,MON,Juan,Pablo Montoya,1975-09-20,Colombian,http://en.wikipedia.org/wiki/Juan_Pablo_Montoya
1,39,karthikeyan,\N,KAR,Narain,Karthikeyan,1977-01-14,Indian,http://en.wikipedia.org/wiki/Narain_Karthikeyan
2,47,baumgartner,\N,\N,Zsolt,Baumgartner,1981-01-01,Hungarian,http://en.wikipedia.org/wiki/Zsolt_Baumgartner
3,61,enge,\N,\N,Tomáš,Enge,1976-09-11,Czech,http://en.wikipedia.org/wiki/Tom%C3%A1%C5%A1_Enge
4,194,salazar,\N,\N,Eliseo,Salazar,1954-11-14,Chilean,http://en.wikipedia.org/wiki/Eliseo_Salazar


In [7]:
races = "SELECT * FROM `formula1.races` LIMIT 5"
client.query(races).to_dataframe()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1100,2023,3,1,Australian Grand Prix,2023-04-02,05:00:00,https://en.wikipedia.org/wiki/2023_Australian_...,2023-03-31,01:30:00,2023-03-31,05:00:00,2023-04-01,01:30:00,2023-04-01,05:00:00,\N,\N
1,1123,2024,3,1,Australian Grand Prix,2024-03-24,04:00:00,https://en.wikipedia.org/wiki/2024_Australian_...,2024-03-22,01:30:00,2024-03-22,05:00:00,2024-03-23,01:30:00,2024-03-23,05:00:00,\N,\N
2,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...,2022-04-08,03:00:00,2022-04-08,06:00:00,2022-04-09,03:00:00,2022-04-09,06:00:00,\N,\N
3,1092,2022,18,22,Japanese Grand Prix,2022-10-09,05:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,2022-10-07,03:00:00,2022-10-07,06:00:00,2022-10-08,03:00:00,2022-10-08,06:00:00,\N,\N
4,1114,2023,16,22,Japanese Grand Prix,2023-09-24,05:00:00,https://en.wikipedia.org/wiki/2023_Japanese_Gr...,2023-09-22,02:30:00,2023-09-22,06:00:00,2023-09-23,02:30:00,2023-09-23,06:00:00,\N,\N


In [8]:
results = "SELECT * FROM `formula1.results` LIMIT 5"
client.query(results).to_dataframe()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,5249,261,71,3,0,2,1,1,1,10.0,65,1:36:14.3,5774300,\N,\N,\N,\N,1
1,5331,264,71,3,0,1,1,1,1,10.0,60,1:30:03.640,5403640,\N,\N,\N,\N,1
2,5415,267,71,3,0,3,1,1,1,10.0,44,1:28:47.1,5327100,\N,\N,\N,\N,1
3,5443,268,71,3,0,3,1,1,1,10.0,53,1:18:03.2,4683200,\N,\N,\N,\N,1
4,5471,269,71,3,0,2,1,1,1,10.0,71,1:45:10.1,6310100,\N,\N,\N,\N,1


## First Query with BigQuery Magics

#### Import and test of BigQuery Magics

In [9]:
%pip install bigquery-magics
%load_ext bigquery_magics
!ipython locate
# %reload_ext bigquery_magics # Ricarica l’estensione bigquery_magics se per esempio hai fatto modifiche o vuoi assicurarti che sia attiva.

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\Alessandro\.ipython


In [10]:
%%bigquery
SELECT * FROM `formula1.results` LIMIT 100

Query is running:   0%|          |

Downloading:   0%|          |

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,5249,261,71,3,0,2,1,1,1,10.0,65,1:36:14.3,5774300,\N,\N,\N,\N,1
1,5331,264,71,3,0,1,1,1,1,10.0,60,1:30:03.640,5403640,\N,\N,\N,\N,1
2,5415,267,71,3,0,3,1,1,1,10.0,44,1:28:47.1,5327100,\N,\N,\N,\N,1
3,5443,268,71,3,0,3,1,1,1,10.0,53,1:18:03.2,4683200,\N,\N,\N,\N,1
4,5471,269,71,3,0,2,1,1,1,10.0,71,1:45:10.1,6310100,\N,\N,\N,\N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,25076,1057,841,51,99,20,11,11,11,0.0,51,+10.254,8026664,44,9,1:45.575,204.696,1
96,25136,1060,847,3,63,8,11,11,11,0.0,70,\N,\N,56,16,1:08.900,225.613,11
97,25276,1067,841,51,99,16,11,11,11,0.0,57,\N,\N,57,13,1:32.904,206.845,11
98,25296,1069,841,51,99,12,11,11,11,0.0,55,\N,\N,44,12,1:41.145,196.221,11


In [11]:
%%bigquery
SELECT * FROM `formula1.races` LIMIT 11

Query is running:   0%|          |

Downloading:   0%|          |

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1100,2023,3,1,Australian Grand Prix,2023-04-02,05:00:00,https://en.wikipedia.org/wiki/2023_Australian_...,2023-03-31,01:30:00,2023-03-31,05:00:00,2023-04-01,01:30:00,2023-04-01,05:00:00,\N,\N
1,1123,2024,3,1,Australian Grand Prix,2024-03-24,04:00:00,https://en.wikipedia.org/wiki/2024_Australian_...,2024-03-22,01:30:00,2024-03-22,05:00:00,2024-03-23,01:30:00,2024-03-23,05:00:00,\N,\N
2,1076,2022,3,1,Australian Grand Prix,2022-04-10,05:00:00,http://en.wikipedia.org/wiki/2022_Australian_G...,2022-04-08,03:00:00,2022-04-08,06:00:00,2022-04-09,03:00:00,2022-04-09,06:00:00,\N,\N
3,1092,2022,18,22,Japanese Grand Prix,2022-10-09,05:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,2022-10-07,03:00:00,2022-10-07,06:00:00,2022-10-08,03:00:00,2022-10-08,06:00:00,\N,\N
4,1114,2023,16,22,Japanese Grand Prix,2023-09-24,05:00:00,https://en.wikipedia.org/wiki/2023_Japanese_Gr...,2023-09-22,02:30:00,2023-09-22,06:00:00,2023-09-23,02:30:00,2023-09-23,06:00:00,\N,\N
5,1124,2024,4,22,Japanese Grand Prix,2024-04-07,05:00:00,https://en.wikipedia.org/wiki/2024_Japanese_Gr...,2024-04-05,02:30:00,2024-04-05,06:00:00,2024-04-06,02:30:00,2024-04-06,06:00:00,\N,\N
6,1142,2024,22,80,Las Vegas Grand Prix,2024-11-23,06:00:00,https://en.wikipedia.org/wiki/2024_Las_Vegas_G...,2024-11-21,02:30:00,2024-11-21,06:00:00,2024-11-22,02:30:00,2024-11-22,06:00:00,\N,\N
7,1125,2024,5,17,Chinese Grand Prix,2024-04-21,07:00:00,https://en.wikipedia.org/wiki/2024_Chinese_Gra...,2024-04-19,03:30:00,2024-04-19,07:30:00,\N,\N,2024-04-20,07:00:00,2024-04-20,03:00:00
8,1119,2023,21,80,Las Vegas Grand Prix,2023-11-19,06:00:00,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...,2023-11-16,04:30:00,2023-11-16,08:00:00,2023-11-17,04:30:00,2023-11-17,08:00:00,\N,\N
9,1137,2024,17,73,Azerbaijan Grand Prix,2024-09-15,11:00:00,https://en.wikipedia.org/wiki/2024_Azerbaijan_...,2024-09-13,09:30:00,2024-09-13,13:00:00,2024-09-14,08:30:00,2024-09-14,12:00:00,\N,\N


In [12]:
%%bigquery
SELECT * FROM `formula1.circuits` LIMIT 11

Query is running:   0%|          |

Downloading:   0%|          |

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,25,galvez,Autódromo Juan y Oscar Gálvez,Buenos Aires,Argentina,-34.6943,-58.45930,8,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Os...
1,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,29,adelaide,Adelaide Street Circuit,Adelaide,Australia,-34.9272,138.61700,58,http://en.wikipedia.org/wiki/Adelaide_Street_C...
3,57,zeltweg,Zeltweg,Styria,Austria,47.2039,14.74780,676,http://en.wikipedia.org/wiki/Zeltweg_Airfield
4,70,red_bull_ring,Red Bull Ring,Spielberg,Austria,47.2197,14.76470,678,http://en.wikipedia.org/wiki/Red_Bull_Ring
5,73,baku,Baku City Circuit,Baku,Azerbaijan,40.3725,49.85330,-7,http://en.wikipedia.org/wiki/Baku_City_Circuit
6,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
7,13,spa,Circuit de Spa-Francorchamps,Spa,Belgium,50.4372,5.97139,401,http://en.wikipedia.org/wiki/Circuit_de_Spa-Fr...
8,40,zolder,Zolder,Heusden-Zolder,Belgium,50.9894,5.25694,36,http://en.wikipedia.org/wiki/Zolder
9,50,nivelles,Nivelles-Baulers,Brussels,Belgium,50.6211,4.32694,139,http://en.wikipedia.org/wiki/Nivelles-Baulers


In [13]:
%%bigquery
SELECT * FROM `formula1.constructors` LIMIT 11

Query is running:   0%|          |

Downloading:   0%|          |

,constructorId,constructorRef,name,nationality,url
0,67,penske,Penske,American,http://en.wikipedia.org/wiki/Penske_Racing
1,73,parnelli,Parnelli,American,http://en.wikipedia.org/wiki/Parnelli
2,88,eagle,Eagle,American,http://en.wikipedia.org/wiki/Anglo_American_Ra...
3,106,scarab,Scarab,American,http://en.wikipedia.org/wiki/Scarab_(constructor)
4,107,watson,Watson,American,http://en.wikipedia.org/wiki/A.J._Watson
5,108,epperly,Epperly,American,http://en.wikipedia.org/wiki/Epperly
6,109,phillips,Phillips,American,http://en.wikipedia.org/wiki/Phillips_(constru...
7,110,lesovsky,Lesovsky,American,http://en.wikipedia.org/wiki/Lesovsky
8,111,trevis,Trevis,American,http://en.wikipedia.org/wiki/Trevis
9,112,meskowski,Meskowski,American,http://en.wikipedia.org/wiki/Meskowski


In [14]:
%%bigquery
SELECT * FROM `formula1.drivers`

Query is running:   0%|          |

Downloading:   0%|          |

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,31,montoya,\N,MON,Juan,Pablo Montoya,1975-09-20,Colombian,http://en.wikipedia.org/wiki/Juan_Pablo_Montoya
1,39,karthikeyan,\N,KAR,Narain,Karthikeyan,1977-01-14,Indian,http://en.wikipedia.org/wiki/Narain_Karthikeyan
2,47,baumgartner,\N,\N,Zsolt,Baumgartner,1981-01-01,Hungarian,http://en.wikipedia.org/wiki/Zsolt_Baumgartner
3,61,enge,\N,\N,Tomáš,Enge,1976-09-11,Czech,http://en.wikipedia.org/wiki/Tom%C3%A1%C5%A1_Enge
4,194,salazar,\N,\N,Eliseo,Salazar,1954-11-14,Chilean,http://en.wikipedia.org/wiki/Eliseo_Salazar
...,...,...,...,...,...,...,...,...,...
856,408,maggs,\N,\N,Tony,Maggs,1937-02-09,South African,http://en.wikipedia.org/wiki/Tony_Maggs
857,433,hansgen,\N,\N,Walt,Hansgen,1919-10-28,American,http://en.wikipedia.org/wiki/Walt_Hansgen
858,654,faulkner,\N,\N,Walt,Faulkner,1920-02-16,American,http://en.wikipedia.org/wiki/Walt_Faulkner
859,771,walt_brown,\N,\N,Walt,Brown,1911-12-30,American,http://en.wikipedia.org/wiki/Walt_Brown_(auto_...


#### Join of the 5 Datasets

In [15]:
%%bigquery
CREATE OR REPLACE TABLE `formula1.joined_results` AS 
SELECT
  c.circuitId AS circuit_id,
  c.name AS circuit_name,
  c.location AS circuit_location,
  c.country AS circuit_country,
  c.lat AS circuit_latitude,
  c.lng AS circuit_longitude,
  c.alt AS circuit_altitude,

  r.raceId AS race_id,
  r.name AS race_name,
  r.year AS race_year,
  r.date AS race_date,

  d.driverId AS driver_id,
  CONCAT(d.forename, ' ', d.surname) AS driver_name,
  d.dob AS driver_dob,
  d.nationality AS driver_nationality,

  cons.constructorId AS constructor_id,
  cons.name AS constructor_name,
  cons.nationality AS constructor_nationality,

  res.resultId AS result_id,
  res.number AS car_number,
  res.grid AS start_grid_position,
  res.position AS final_position,
  res.points AS race_points,
  res.laps AS total_laps,
  res.fastestLap AS fastest_lap_number,
  res.fastestLapTime AS fastest_lap_time,
  res.fastestLapSpeed AS fastest_lap_speed

FROM `formula1.results` res
JOIN `formula1.races` r
  ON res.raceId = r.raceId
JOIN `formula1.circuits` c
  ON r.circuitId = c.circuitId
JOIN `formula1.drivers` d
  ON res.driverId = d.driverId
JOIN `formula1.constructors` cons
  ON res.constructorId = cons.constructorId

Query is running:   0%|          |

""


## Pandas Loading of the Joined Dataset

In [16]:
%%bigquery df_f1
SELECT * FROM `formula1.joined_results`

Query is running:   0%|          |

Downloading:   0%|          |

In [17]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,constructor_nationality,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed
0,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,Italian,13775,2,1,1,9.0,80,\N,\N,\N
1,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13790,11,3,\N,0.0,3,\N,\N,\N
2,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13787,3,11,\N,0.0,34,\N,\N,\N
3,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13791,34,18,\N,0.0,2,\N,\N,\N
4,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13784,10,6,10,0.0,77,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,French,26341,31,15,16,0.0,57,45,1:21.354,233.557
26749,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,British,26333,14,10,8,4.0,58,52,1:20.493,236.055
26750,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,Austrian,26344,1,1,\N,0.0,3,3,1:23.115,228.608
26751,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,British,26329,81,5,4,12.0,58,1,1:20.199,236.920


## Cleaning the Joined dataset

In [18]:
df_f1.dtypes

circuit_id                   Int64
circuit_name                object
circuit_location            object
circuit_country             object
circuit_latitude           float64
circuit_longitude          float64
circuit_altitude             Int64
race_id                      Int64
race_name                   object
race_year                    Int64
race_date                   dbdate
driver_id                    Int64
driver_name                 object
driver_dob                  dbdate
driver_nationality          object
constructor_id               Int64
constructor_name            object
constructor_nationality     object
result_id                    Int64
car_number                   Int64
start_grid_position          Int64
final_position              object
race_points                float64
total_laps                   Int64
fastest_lap_number          object
fastest_lap_time            object
fastest_lap_speed           object
dtype: object

In [19]:
# dtypes conversion
df_f1['final_position'] = pd.to_numeric(df_f1['final_position'], errors='coerce')
df_f1['fastest_lap_number'] = pd.to_numeric(df_f1['fastest_lap_number'], errors='coerce')
df_f1['fastest_lap_speed'] = (
    df_f1['fastest_lap_speed']
    .str.replace(',', '.', regex=False)  # sostituisci virgole con punti
)
df_f1['fastest_lap_speed'] = pd.to_numeric(df_f1['fastest_lap_speed'], errors='coerce')
# coerce converte ignorando gli errori ('\\N' diventa NaN)

In [20]:
df_f1.dtypes

circuit_id                   Int64
circuit_name                object
circuit_location            object
circuit_country             object
circuit_latitude           float64
circuit_longitude          float64
circuit_altitude             Int64
race_id                      Int64
race_name                   object
race_year                    Int64
race_date                   dbdate
driver_id                    Int64
driver_name                 object
driver_dob                  dbdate
driver_nationality          object
constructor_id               Int64
constructor_name            object
constructor_nationality     object
result_id                    Int64
car_number                   Int64
start_grid_position          Int64
final_position             float64
race_points                float64
total_laps                   Int64
fastest_lap_number         float64
fastest_lap_time            object
fastest_lap_speed          float64
dtype: object

In [21]:
df_f1.describe()

,circuit_id,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_year,driver_id,constructor_id,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_speed
count,26753.0,26753.000000,26753.000000,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,26753.0,15806.000000,26753.000000,26753.0,8252.000000,8252.000000
mean,23.814077,33.949189,5.724360,276.893545,551.646656,1991.400964,278.639069,50.165589,13379.899151,18.153927,11.137293,8.020499,1.988078,46.312152,42.732913,204.116330
std,19.10885,25.244714,57.639049,409.12065,313.28841,19.950263,282.723999,61.544163,7726.652173,15.581135,7.201737,4.840796,4.351595,29.491712,16.603460,21.377265
min,1.0,-37.849700,-118.189000,-7.0,1.0,1950.0,1.0,1.0,1.0,0.0,0.0,1.000000,0.000000,0.0,1.000000,89.540000
25%,9.0,34.843100,-1.016940,18.0,300.0,1977.0,57.0,6.0,6689.0,7.0,5.0,4.000000,0.000000,23.0,33.000000,193.322500
50%,18.0,43.734700,6.947500,153.0,531.0,1991.0,172.0,25.0,13377.0,16.0,11.0,8.000000,0.000000,53.0,46.000000,204.894000
75%,34.0,49.254200,14.764700,401.0,811.0,2009.0,399.0,63.0,20070.0,24.0,17.0,11.000000,2.000000,66.0,54.000000,217.324000
max,80.0,57.265300,144.968000,2227.0,1144.0,2024.0,862.0,215.0,26764.0,208.0,34.0,33.000000,50.000000,200.0,85.000000,257.320000


In [22]:
from google.cloud import bigquery
client = bigquery.Client()
table_id = "formula1.joined_results"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(df_f1, table_id, job_config=job_config) # Crea una configurazione di caricamento (LoadJobConfig) dove:
# write_disposition="WRITE_TRUNCATE" dice a BigQuery:
# sovrascrive completamente la tabella esistente (TRUNCATE = tronca).
job.result()  

LoadJob<project=ccbd-exam-2025-risino, location=EU, id=3f6c2805-80af-4f2c-9053-9ebd8b7663f8>

In [23]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,constructor_nationality,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed
0,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,Italian,13775,2,1,1.0,9.0,80,NaN,\N,NaN
1,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13790,11,3,NaN,0.0,3,NaN,\N,NaN
2,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13787,3,11,NaN,0.0,34,NaN,\N,NaN
3,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13791,34,18,NaN,0.0,2,NaN,\N,NaN
4,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,British,13784,10,6,10.0,0.0,77,NaN,\N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,French,26341,31,15,16.0,0.0,57,45.0,1:21.354,233.557
26749,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,British,26333,14,10,8.0,4.0,58,52.0,1:20.493,236.055
26750,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,Austrian,26344,1,1,NaN,0.0,3,3.0,1:23.115,228.608
26751,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,British,26329,81,5,4.0,12.0,58,1.0,1:20.199,236.920


In [24]:
%%bigquery df_f1
SELECT *, IF(final_position = 1, 1, 0) AS wins
FROM formula1.joined_results

Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
from google.cloud import bigquery
client = bigquery.Client()
table_id = "formula1.joined_results"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
job = client.load_table_from_dataframe(df_f1, table_id, job_config=job_config) # Crea una configurazione di caricamento (LoadJobConfig) dove:
# write_disposition="WRITE_TRUNCATE" dice a BigQuery:
# sovrascrive completamente la tabella esistente (TRUNCATE = tronca).
job.result()  

LoadJob<project=ccbd-exam-2025-risino, location=EU, id=a634c459-153b-4481-97e3-603250c3eeaf>

In [26]:
display(df_f1)

,circuit_id,circuit_name,circuit_location,circuit_country,circuit_latitude,circuit_longitude,circuit_altitude,race_id,race_name,race_year,...,result_id,car_number,start_grid_position,final_position,race_points,total_laps,fastest_lap_number,fastest_lap_time,fastest_lap_speed,wins
0,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,13775,2,1,1.0,9.0,80,NaN,\N,NaN,1
1,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,13790,11,3,NaN,0.0,3,NaN,\N,NaN,0
2,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,13787,3,11,NaN,0.0,34,NaN,\N,NaN,0
3,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,13791,34,18,NaN,0.0,2,NaN,\N,NaN,0
4,43,Long Beach,California,USA,33.7651,-118.189,12,562,United States Grand Prix West,1976,...,13784,10,6,10.0,0.0,77,NaN,\N,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26748,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,26341,31,15,16.0,0.0,57,45.0,1:21.354,233.557,0
26749,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,26333,14,10,8.0,4.0,58,52.0,1:20.493,236.055,0
26750,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,26344,1,1,NaN,0.0,3,3.0,1:23.115,228.608,0
26751,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1123,Australian Grand Prix,2024,...,26329,81,5,4.0,12.0,58,1.0,1:20.199,236.920,0


In [27]:
df_f1["wins"].value_counts()

wins
0    25625
1     1128
Name: count, dtype: Int64

In [28]:
df_f1["race_id"].nunique()

1125

In [29]:
%%bigquery 
SELECT wins, count(*) as wins_count
FROM formula1.joined_results
GROUP BY wins

Query is running:   0%|          |

Downloading:   0%|          |

,wins,wins_count
0,1,1128
1,0,25625


## Three main Queries

In [30]:
%%bigquery
SELECT
  driver_name,
  driver_nationality,
  COUNT(*) AS total_wins
FROM
  formula1.joined_results
WHERE
  wins = 1
GROUP BY
  driver_name, driver_nationality
ORDER BY
  total_wins DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,driver_name,driver_nationality,total_wins
0,Lewis Hamilton,British,105
1,Michael Schumacher,German,91
2,Max Verstappen,Dutch,63
3,Sebastian Vettel,German,53
4,Alain Prost,French,51
...,...,...,...
110,Lorenzo Bandini,Italian,1
111,Vittorio Brambilla,Italian,1
112,Heikki Kovalainen,Finnish,1
113,Esteban Ocon,French,1


In [31]:
%%bigquery
SELECT constructor_name,
  COUNT(*) AS total_constructor_wins
FROM
  formula1.joined_results
WHERE
  wins = 1
GROUP BY
  constructor_name
ORDER BY
  total_constructor_wins DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,total_constructor_wins
0,Ferrari,249
1,McLaren,185
2,Mercedes,129
3,Red Bull,122
4,Williams,114
5,Team Lotus,45
6,Renault,35
7,Benetton,27
8,Brabham,23
9,Tyrrell,23


In [32]:
%%bigquery
SELECT
  circuit_name,
  circuit_country,
  MAX(fastest_lap_speed) AS max_fastest_lap_speed
FROM
  formula1.joined_results
WHERE
  fastest_lap_speed IS NOT NULL
GROUP BY
  circuit_name, circuit_country
ORDER BY
  max_fastest_lap_speed DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,circuit_name,circuit_country,max_fastest_lap_speed
0,Autodromo Nazionale di Monza,Italy,257.320
1,Jeddah Corniche Circuit,Saudi Arabia,244.962
2,Silverstone Circuit,UK,243.494
3,Circuit de Spa-Francorchamps,Belgium,240.822
4,Autodromo Internazionale del Mugello,Italy,239.518
5,Albert Park Grand Prix Circuit,Australia,238.066
6,Red Bull Ring,Austria,236.894
7,Losail International Circuit,Qatar,236.798
8,Las Vegas Strip Street Circuit,United States,235.292
9,Autodromo Enzo e Dino Ferrari,Italy,234.121


## To visualize the details of the results click on the link below
[Open my Looker Studio Dashboard](https://lookerstudio.google.com/reporting/5d3818d5-e596-4fe2-bfea-6e9cb4a5e3ec)

## Data Enrichment
[To see the Databricks side with the Machine Learning in-depth analysis click here](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3706853530635392/2578662843473137/2314117943557060/latest.html)

## 3rd Query

In [42]:
%%bigquery
SELECT
  constructor_name,
  COUNT(DISTINCT race_id) AS num_races,
  COUNT(driver_id) AS total_pilot_participation
FROM
  `formula1.joined_results`
WHERE
  constructor_name IN ('Ferrari', 'McLaren')
GROUP BY
  constructor_name;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_races,total_pilot_participation
0,Ferrari,1100,2439
1,McLaren,929,1923


In [44]:
%%bigquery
WITH piloti_per_gara AS (
  SELECT
    constructor_name,
    race_id,
    COUNT(DISTINCT driver_id) AS num_pilots
  FROM
    `formula1.joined_results`
  WHERE
    constructor_name IN ('Ferrari', 'McLaren')
  GROUP BY
    constructor_name,
    race_id
)

SELECT
  constructor_name,
  num_pilots,
  COUNT(*) AS count_races
FROM
  piloti_per_gara
GROUP BY
  constructor_name,
  num_pilots
ORDER BY
  constructor_name,
  num_pilots;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_pilots,count_races
0,Ferrari,1,39
1,Ferrari,2,948
2,Ferrari,3,41
3,Ferrari,4,34
4,Ferrari,5,20
5,Ferrari,6,10
6,Ferrari,7,4
7,Ferrari,8,2
8,Ferrari,9,1
9,Ferrari,10,1


In [37]:
%%bigquery
SELECT
  COUNT(*) AS total_races_not_2_pilots
FROM (
  SELECT
    race_year,
    race_name,
    constructor_name,
    COUNT(DISTINCT driver_id) AS num_pilots
  FROM
    `formula1.joined_results`
  GROUP BY
    race_year,
    race_name,
    constructor_name
  HAVING
    num_pilots != 2
)

Query is running:   0%|          |

Downloading:   0%|          |

,total_races_not_2_pilots
0,2598


In [38]:
%%bigquery
SELECT
  constructor_name,
  COUNT(*) AS num_races_not_2_pilots
FROM (
  SELECT
    race_year,
    race_name,
    constructor_name,
    COUNT(DISTINCT driver_id) AS num_pilots
  FROM
    `formula1.joined_results`
  GROUP BY
    race_year,
    race_name,
    constructor_name
  HAVING
    num_pilots != 2
)
GROUP BY
  constructor_name
ORDER BY
  num_races_not_2_pilots DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,constructor_name,num_races_not_2_pilots
0,BRM,153
1,Ferrari,152
2,Ensign,112
3,March,106
4,Osella,92
...,...,...
168,Connew,1
169,Cooper-Alfa Romeo,1
170,RE,1
171,Marussia,1


In [41]:
%%bigquery
CREATE OR REPLACE TABLE `formula1.constructor_unusual_pilots_view` AS
SELECT
  race_year,
  race_name,
  constructor_name,
  COUNT(DISTINCT driver_id) AS num_pilots,
  STRING_AGG(DISTINCT driver_name, ', ') AS pilots_list
FROM
  `formula1.joined_results`
GROUP BY
  race_year,
  race_name,
  constructor_name
HAVING
  num_pilots != 2

Query is running:   0%|          |

""


In [47]:
!findstr "project_id" "%GOOGLE_APPLICATION_CREDENTIALS%"

  "project_id": "ccbd-exam-2025-risino",
